In [28]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

artists_data_path = "../data/artists.jsonl"
sessions_data_path = "../data/sessions.jsonl"
track_storage_data_path = "../data/track_storage.jsonl"
tracks_data_path = "../data/tracks.jsonl"
users_data_path = "../data/users.jsonl"

artists_data_frame = pd.read_json(artists_data_path, lines=True)
sessions_data_frame = pd.read_json(sessions_data_path, lines=True)
tracks_data_frame = pd.read_json(tracks_data_path, lines=True)
users_data_frame = pd.read_json(users_data_path, lines=True)

# Mergowanie podobnych gatunków muzycznych (artists)
TODO


# Scalenie danych

In [29]:
merged_data_frame = pd.merge(sessions_data_frame, tracks_data_frame, left_on="track_id",
                             right_on="id")
merged_data_frame = pd.merge(merged_data_frame, users_data_frame, left_on="user_id",
                             right_on="user_id")
merged_data_frame = pd.merge(merged_data_frame, artists_data_frame, left_on="id_artist",
                             right_on="id")
merged_data_frame.head(5)


,timestamp,user_id,track_id,event_type,session_id,id_x,name_x,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,name_y,city,street,favourite_genres,premium_user,id_y,name,genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,PLAY,124,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Klara Herdzik,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,PLAY,151,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Szymon Zawal,Gdynia,al. Głogowa 14/10,"[filmi, regional mexican, folk]",False,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,PLAY,544,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,PLAY,534,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,PLAY,547,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"


## Wyliczenie, czy dana piosenka w danej sesji została pominięta

In [30]:
# delete rows with event_type other than PLAY and SKIP
merged_data_frame = merged_data_frame[merged_data_frame["event_type"].isin(["PLAY", "SKIP"])]

# TODO moze to trzeba liczyc inaczej?

# if for one user_id and track_id and session_id there is only PLAY event, then the track was not skipped
merged_data_frame["skipped"] = merged_data_frame.groupby(["user_id", "track_id", "session_id"])[
  "event_type"].transform(lambda x: x != "PLAY").astype(bool)

# if there are 2 rows with the same user_id, track_id and session_id, and different "skipped" value, remove the row with "skipped" = 0
merged_data_frame = merged_data_frame.drop_duplicates(subset=["user_id", "track_id", "session_id"],
                                                      keep="last")

## Usunięcie niepotrzebnych kolumn

In [31]:
merged_data_frame = merged_data_frame.drop(
  columns=["id_x", "id_y", "premium_user", "name_x", "name_y", "city", "street", "id_artist",
           "event_type", "timestamp", "user_id", "track_id", "session_id"])
merged_data_frame.head(10)

,popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,favourite_genres,name,genres,skipped
0,34,247707,0,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[permanent wave, mandopop, funk]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False
1,34,247707,0,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[filmi, regional mexican, folk]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False
2,34,247707,0,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[psychedelic rock, country rock, rock en espanol]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False
3,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[psychedelic rock, country rock, rock en espanol]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False
4,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[psychedelic rock, country rock, rock en espanol]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False
5,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[latin alternative, alternative rock, classic rock]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False
7,21,248360,0,1972-07-21,0.513,0.576,7,-9.704,0.0384,0.01130,0.000035,0.248,0.662,122.853,"[soft rock, hoerspiel, ranchera]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",True
9,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[soft rock, hoerspiel, ranchera]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",True
12,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[soft rock, hoerspiel, ranchera]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",True
15,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[soft rock, hoerspiel, ranchera]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",True


In [32]:
merged_data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9881 entries, 0 to 16822
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   popularity        9881 non-null   int64  
 1   duration_ms       9881 non-null   int64  
 2   explicit          9881 non-null   int64  
 3   release_date      9881 non-null   object 
 4   danceability      9881 non-null   float64
 5   energy            9881 non-null   float64
 6   key               9881 non-null   int64  
 7   loudness          9881 non-null   float64
 8   speechiness       9881 non-null   float64
 9   acousticness      9881 non-null   float64
 10  instrumentalness  9881 non-null   float64
 11  liveness          9881 non-null   float64
 12  valence           9881 non-null   float64
 13  tempo             9881 non-null   float64
 14  favourite_genres  9881 non-null   object 
 15  name              9881 non-null   object 
 16  genres            9881 non-null   object 
 17 

In [33]:
merged_data_frame.describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,9881.000000,9.881000e+03,9881.000000,9881.000000,9881.000000,9881.00000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000
mean,44.646594,2.310971e+05,0.080963,0.563325,0.600228,5.25463,-9.125568,0.083663,0.328242,0.067860,0.207252,0.539190,120.525376
std,25.085211,9.345541e+04,0.272793,0.165085,0.230087,3.55038,4.286563,0.121603,0.311197,0.199239,0.192774,0.250657,28.640296
min,0.000000,3.100000e+04,0.000000,0.064000,0.000592,0.00000,-35.898000,0.022700,0.000001,0.000000,0.016000,0.002280,36.241000
25%,25.000000,1.803480e+05,0.000000,0.453000,0.428000,2.00000,-11.515000,0.033500,0.042500,0.000000,0.095100,0.338000,98.231000
50%,39.000000,2.158000e+05,0.000000,0.570000,0.624000,5.00000,-8.446000,0.042800,0.223000,0.000044,0.127000,0.544000,120.031000
75%,75.000000,2.626530e+05,0.000000,0.683000,0.794000,9.00000,-6.114000,0.077100,0.579000,0.005030,0.251000,0.742000,138.023000
max,96.000000,2.365307e+06,1.000000,0.975000,0.998000,11.00000,1.716000,0.965000,0.996000,0.978000,0.997000,0.991000,209.231000


In [44]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import metrics
from scipy.stats import entropy

# Combine genres and favourite_genres
all_genres = list(merged_data_frame['favourite_genres'] + merged_data_frame['genres'])

# One-hot encode the genres
mlb = MultiLabelBinarizer()
encoded_all_genres = mlb.fit_transform(all_genres)

# Split encoded_genres into genres and favourite_genres
encoded_favourite_genres = encoded_all_genres[:, :len(merged_data_frame['favourite_genres'][0])]
encoded_genres = encoded_all_genres[:, len(merged_data_frame['favourite_genres'][0]):]

print(encoded_favourite_genres.shape)
for fav_genre in encoded_favourite_genres:
    print(fav_genre)
print(encoded_genres.shape)

# Add encoded genres to the dataframe
df = merged_data_frame.join(pd.DataFrame(encoded_genres, columns=mlb.classes_))
df = df.join(pd.DataFrame(encoded_favourite_genres, columns=mlb.classes_))

print("Entropy of skipped column: ", str(round(entropy(merged_data_frame["skipped"].value_counts()), 4)))

print("MI(skipped, genres): ", str(round(metrics.mutual_info_score(merged_data_frame["skipped"], merged_data_frame["genres"]), 4)))

df.head(5)

(9881, 3)
[[0 0 0]
 [0 0 0]
 [0 0 0]
 ...
 [0 0 0]
 [0 0 0]
 [0 0 0]]
(9881, 1763)


ValueError: Shape of passed values is (9881, 1763), indices imply (9881, 1766)

In [35]:
# save to jsonl file
merged_data_frame.to_json("../data/merged_data.jsonl", orient="records", lines=True)
